In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
import matplotlib.pyplot as plt
import numpy as np
import os

In [2]:
# 超参数设置
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
LEARNING_RATE = 2e-4
BATCH_SIZE = 128
IMAGE_SIZE = 64
CHANNELS_IMG = 1
NOISE_DIM = 100
NUM_EPOCHS = 50
FEATURES_DISC = 64
FEATURES_GEN = 64

In [3]:
# 数据预处理
transform = transforms.Compose([
    transforms.Resize(IMAGE_SIZE),
    transforms.ToTensor(),
    transforms.Normalize(
        [0.5 for _ in range(CHANNELS_IMG)], 
        [0.5 for _ in range(CHANNELS_IMG)]
    ),
])

# 加载MNIST数据集
dataset = datasets.MNIST(root="dataset/", train=True, transform=transform, download=True)
dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)

Failed to download (trying next):
HTTP Error 404: Not Found



100.0%


Extracting dataset/MNIST/raw/train-images-idx3-ubyte.gz to dataset/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



100.0%


Extracting dataset/MNIST/raw/train-labels-idx1-ubyte.gz to dataset/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



100.0%


Extracting dataset/MNIST/raw/t10k-images-idx3-ubyte.gz to dataset/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



100.0%

Extracting dataset/MNIST/raw/t10k-labels-idx1-ubyte.gz to dataset/MNIST/raw



In [4]:
class Generator(nn.Module):
    def __init__(self, noise_dim, channels_img, features_gen):
        super(Generator, self).__init__()
        self.net = nn.Sequential(
            # 输入: N x noise_dim x 1 x 1
            self._block(noise_dim, features_gen * 16, 4, 1, 0),  # 4x4
            self._block(features_gen * 16, features_gen * 8, 4, 2, 1),  # 8x8
            self._block(features_gen * 8, features_gen * 4, 4, 2, 1),  # 16x16
            self._block(features_gen * 4, features_gen * 2, 4, 2, 1),  # 32x32
            nn.ConvTranspose2d(
                features_gen * 2, channels_img, kernel_size=4, stride=2, padding=1
            ),
            # 输出: N x channels_img x 64 x 64
            nn.Tanh(),
        )

    def _block(self, in_channels, out_channels, kernel_size, stride, padding):
        return nn.Sequential(
            nn.ConvTranspose2d(
                in_channels,
                out_channels,
                kernel_size,
                stride,
                padding,
                bias=False,
            ),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(),
        )

    def forward(self, x):
        return self.net(x)

In [5]:
class Discriminator(nn.Module):
    def __init__(self, channels_img, features_disc):
        super(Discriminator, self).__init__()
        self.disc = nn.Sequential(
            # 输入: N x channels_img x 64 x 64
            nn.Conv2d(
                channels_img, features_disc, kernel_size=4, stride=2, padding=1
            ),  # 32x32
            nn.LeakyReLU(0.2),
            self._block(features_disc, features_disc * 2, 4, 2, 1),  # 16x16
            self._block(features_disc * 2, features_disc * 4, 4, 2, 1),  # 8x8
            self._block(features_disc * 4, features_disc * 8, 4, 2, 1),  # 4x4
            nn.Conv2d(features_disc * 8, 1, kernel_size=4, stride=2, padding=0),  # 1x1
            nn.Sigmoid(),
        )

    def _block(self, in_channels, out_channels, kernel_size, stride, padding):
        return nn.Sequential(
            nn.Conv2d(
                in_channels,
                out_channels,
                kernel_size,
                stride,
                padding,
                bias=False,
            ),
            nn.BatchNorm2d(out_channels),
            nn.LeakyReLU(0.2),
        )

    def forward(self, x):
        return self.disc(x)

In [6]:
# 初始化生成器和判别器
gen = Generator(NOISE_DIM, CHANNELS_IMG, FEATURES_GEN).to(device)
disc = Discriminator(CHANNELS_IMG, FEATURES_DISC).to(device)

# 初始化权重
def weights_init(m):
    if isinstance(m, (nn.Conv2d, nn.ConvTranspose2d, nn.BatchNorm2d)):
        nn.init.normal_(m.weight.data, 0.0, 0.02)

gen.apply(weights_init)
disc.apply(weights_init)

# 初始化优化器
opt_gen = optim.Adam(gen.parameters(), lr=LEARNING_RATE, betas=(0.5, 0.999))
opt_disc = optim.Adam(disc.parameters(), lr=LEARNING_RATE, betas=(0.5, 0.999))

# 损失函数
criterion = nn.BCELoss()

# 用于生成样本的固定噪声
fixed_noise = torch.randn(32, NOISE_DIM, 1, 1).to(device)

# Tensorboard
writer_real = SummaryWriter(f"logs/real")
writer_fake = SummaryWriter(f"logs/fake")
step = 0

In [ ]:
gen.train()
disc.train()

for epoch in range(NUM_EPOCHS):
    for batch_idx, (real, _) in enumerate(dataloader):
        real = real.to(device)
        noise = torch.randn(BATCH_SIZE, NOISE_DIM, 1, 1).to(device)
        fake = gen(noise)
        
        ### 训练判别器: max log(D(x)) + log(1 - D(G(z)))
        disc_real = disc(real).view(-1)
        loss_disc_real = criterion(disc_real, torch.ones_like(disc_real))
        disc_fake = disc(fake.detach()).view(-1)
        loss_disc_fake = criterion(disc_fake, torch.zeros_like(disc_fake))
        loss_disc = (loss_disc_real + loss_disc_fake) / 2
        disc.zero_grad()
        loss_disc.backward()
        opt_disc.step()
        
        ### 训练生成器: min log(1 - D(G(z))) <-> max log(D(G(z)))
        output = disc(fake).view(-1)
        loss_gen = criterion(output, torch.ones_like(output))
        gen.zero_grad()
        loss_gen.backward()
        opt_gen.step()
        
        # 打印训练状态
        if batch_idx % 100 == 0:
            print(
                f"Epoch [{epoch}/{NUM_EPOCHS}] Batch {batch_idx}/{len(dataloader)} \
                  Loss D: {loss_disc:.4f}, loss G: {loss_gen:.4f}"
            )
            
            with torch.no_grad():
                fake = gen(fixed_noise)
                # 取部分样本
                img_grid_real = torchvision.utils.make_grid(
                    real[:32], normalize=True
                )
                img_grid_fake = torchvision.utils.make_grid(
                    fake[:32], normalize=True
                )
                
                writer_real.add_image("Real", img_grid_real, global_step=step)
                writer_fake.add_image("Fake", img_grid_fake, global_step=step)
                
            step += 1

Epoch [0/50] Batch 0/469                   Loss D: 0.6936, loss G: 0.7839


In [ ]:
# 保存模型
def save_models(gen, disc, epoch):
    torch.save(gen.state_dict(), f"gen_epoch_{epoch}.pth")
    torch.save(disc.state_dict(), f"disc_epoch_{epoch}.pth")
    print(f"Models saved at epoch {epoch}")

# 加载模型
def load_models(gen, disc, epoch):
    gen.load_state_dict(torch.load(f"gen_epoch_{epoch}.pth"))
    disc.load_state_dict(torch.load(f"disc_epoch_{epoch}.pth"))
    print(f"Models loaded from epoch {epoch}")

In [ ]:
def plot_generated_images(gen, num_images=25):
    noise = torch.randn(num_images, NOISE_DIM, 1, 1).to(device)
    gen.eval()
    with torch.no_grad():
        generated_images = gen(noise).cpu().numpy()
    
    fig, axes = plt.subplots(5, 5, figsize=(10, 10))
    for i, ax in enumerate(axes.flat):
        ax.imshow(generated_images[i, 0, :, :], cmap='gray')
        ax.axis('off')
    plt.tight_layout()
    plt.show()

# 训练完成后调用
plot_generated_images(gen)

In [ ]:
if __name__ == "__main__":
    # 训练模型
    for epoch in range(NUM_EPOCHS):
        # 训练代码如上...
        
        # 每5个epoch保存一次模型
        if epoch % 5 == 0:
            save_models(gen, disc, epoch)
    
    # 训练完成后生成样本
    plot_generated_images(gen)